In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from pyspark.sql.functions import trim

import time

t0 = time.time()


# New API
spark_session = SparkSession.builder\
        .master("spark://192.168.2.156:7077") \
        .appName("Abbe group 7 test code")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 4)\
        .config("spark.dynamicAllocation.maxExecutors", 4)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

In [ ]:
#reading in the reddit data from the corpus which was the suggested dataset
df = spark_session.read.json("hdfs://192.168.2.156:9000/data/reddit/corpus-webis-tldr-17.json")

In [ ]:
# get the top five contributers
df_subreddit = df.groupBy(df.subreddit).count().orderBy("count", ascending = False).limit(5)
df_subreddit.show()

In [ ]:
# filter for just subreddits
df_filtered = df.join(df_subreddit, on="subreddit", how="inner")

In [ ]:
# group by author and subreddit and count posts per author
df_authors = df_filtered.groupBy("subreddit", "author").count()

In [ ]:
from pyspark.sql import functions as F

# take the top 5 contributers for each subreddit
subreddits = [row["subreddit"] for row in df_subreddit.collect()]  
top_authors_list = []

for subreddit in subreddits:
    df_top = (
        df_authors.filter((F.col("subreddit") == subreddit) & (F.col("author") != "[deleted]"))  # filter for one subreddit at a time
        .orderBy(F.desc("count"))  # sort
        .limit(5) 
    )
    top_authors_list.append(df_top)

# join the results together
df_top_authors = top_authors_list[0]
for df_top in top_authors_list[1:]:
    df_top_authors = df_top_authors.union(df_top)

df_top_authors.show()

In [ ]:
t1 = time.time()

total = t1-t0

print("Total time", total)

In [7]:
spark_session.stop()